In [ ]:
!pip install boto3

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
prefix = 'feature-store/amazon-reviews/balanced-raw-with-header'

df_balanced_raw_with_header_s3_uri = 's3://{}/{}/data.csv'.format(bucket, prefix)

print(df_balanced_raw_with_header_s3_uri)

In [ ]:
prefix = 'model/amazon-reviews/autopilot'

autopilot_model_output_s3_uri = 's3://{}/{}/data.csv'.format(bucket, prefix)

print(autopilot_model_output_s3_uri)


In [ ]:
max_candidates = 10

job_config = {
    'CompletionCriteria': {
      'MaxRuntimePerTrainingJobInSeconds': 600,
      'MaxCandidates': max_candidates,
      'MaxAutoMLJobRuntimeInSeconds': 3600
    },
}

input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': '{}'.format(df_balanced_raw_with_header_s3_uri)
        }
      },
      'TargetAttributeName': 'is_positive_sentiment'
    }
]

output_data_config = {
    'S3OutputPath': '{}'.format(autopilot_model_output_s3_uri)
}

#problem_type = 'Regression'

#auto_ml_job_objective = {
#    'MetricName': 'MSE'
#}

## Launching the SageMaker AutoPilot job

We can now launch the job by calling the `create_auto_ml_job` API.

In [ ]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-dm-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

In [ ]:
sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig=job_config,
#                      ProblemType=problem_type,
#                      AutoMLJobObjective=auto_ml_job_objective,
                      RoleArn=role)

### Tracking the progress of the AutoPilot job
SageMaker AutoPilot job consists of four high-level steps : 
* Data Preprocessing, where the dataset is split into train and validation sets.
* Recommending Pipelines, where the dataset is analyzed and SageMaker AutoPilot comes up with a list of ML pipelines that should be tried out on the dataset.
* Automatic Feature Engineering, where SageMaker AutoPilot performs feature transformation on individual features of the dataset as well as at an aggregate level.
* ML pipeline selection and hyperparameter tuning, where the top performing pipeline is selected along with the optimal hyperparameters for the training algorithm (the last stage of the pipeline). 

In [ ]:
# Sleep for a bit to ensure the AutoML job above has time to start
import time
time.sleep(3)

job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']

if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('AnalyzingData'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Data analysis complete")
    
print(job)

In [ ]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('FeatureEngineering'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Feature engineering complete")
    
print(job)

In [ ]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('ModelTuning'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Model tuning complete")
    
print(job)

### Viewing all candidates explored by SageMaker AutoPilot
Once model tuning is complete, you can view all the candidates (pipeline evaluations with different hyperparameter combinations) that were explored by AutoML and sort them by their final performance metric.

In [ ]:
candidates = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, 
                                                SortBy='FinalObjectiveMetricValue')['Candidates']
for index, candidate in enumerate(candidates):
    print(str(index) + "  " 
        + candidate['CandidateName'] + "  " 
        + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))

## Inspect SageMaker AutoPilot trials with Amazon SageMaker Experiments
SageMaker AutoPilot automatically creates a new experiment, and pushes information for each trial. 

In [ ]:
from sagemaker.analytics import ExperimentAnalytics, TrainingJobAnalytics

exp = ExperimentAnalytics(
    sagemaker_session=sess, 
    experiment_name=auto_ml_job_name + '-aws-auto-ml-job',
)

df = exp.dataframe()
df

## Viewing notebooks generated by SageMaker AutoPilot
Once data analysis is complete, SageMaker AutoPilot generates two notebooks: 
* Data exploration,
* Candidate definition.

In [ ]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print(job)

### Let's copy all of the generated resources including the two notebooks.

In [ ]:
generated_resources = job['AutoMLJobArtifacts']['DataExplorationNotebookLocation'].rstrip('notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb')
generated_resources

In [ ]:
!rm -rf ./generated_module
!rm -rf ./notebooks
!aws s3 cp --recursive $generated_resources .

### In the file view, open the `notebooks/` and `generated_module/` folders.  Lots of useful information in there!

## Deploying the best candidate
Now that we have successfully completed the AutoML job on our dataset and visualized the trials, we can create a model from any of the trials with a single API call and then deploy that model for online or batch prediction using [Inference Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html). For this notebook, we deploy only the best performing trial for inference.

The best candidate is the one we're really interested in.

In [ ]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_identifier = best_candidate['CandidateName']

print("Candidate name: " + best_candidate_identifier)
print("Metric name: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("Metric value: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

In [ ]:
best_candidate

We can see the containers and models composing the Inference Pipeline.

In [ ]:
for container in best_candidate['InferenceContainers']:
    print(container['Image'])
    print(container['ModelDataUrl'])
    print('======================')

In [ ]:
model_name = 'automl-dm-model-' + timestamp_suffix

model_arn = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

print('Best candidate model ARN: ', model_arn['ModelArn'])

Let's deploy the pipeline.

In [ ]:
# EndpointConfig name
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
epc_name = 'automl-dm-epc-' + timestamp_suffix

# Endpoint name
ep_name = 'automl-dm-ep-' + timestamp_suffix
variant_name = 'automl-dm-variant-' + timestamp_suffix

print(ep_name)
print(variant_name)

In [ ]:
ep_config = sm.create_endpoint_config(EndpointConfigName = epc_name,
                                      ProductionVariants=[{'InstanceType':'ml.m4.xlarge',
                                                           'InitialInstanceCount':1,
                                                           'ModelName':model_name,
                                                           'VariantName':variant_name}])

create_endpoint_response = sm.create_endpoint(EndpointName=ep_name,
                                              EndpointConfigName=epc_name)
print(create_endpoint_response['EndpointArn'])

In [ ]:
%%time
sm.get_waiter('endpoint_in_service').wait(EndpointName=ep_name)

In [ ]:
resp = sm.describe_endpoint(EndpointName=ep_name)
status = resp['EndpointStatus']

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

## Scoring the Best Candidate
Let's predict and score the validation set. We'll compute metrics ourselves just for fun.

In [ ]:
sm_rt = boto3.Session().client('runtime.sagemaker', region_name=region)

In [ ]:
#ep_name = ''

csv_line_predict_positive = """I loved it!  I wish there was a new season..."""
response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_positive)

response_body = response['Body'].read().decode("utf-8").strip()
response_body

In [ ]:
csv_line_predict_negative = """This isn't good.  Complete waste of time."""
response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_negative)

response_body = response['Body'].read().decode("utf-8").strip()
response_body

In [ ]:
for review_body in test['review_body']:
    print(review_body)

In [ ]:
test['review_body'].shape

In [ ]:
responses = [int(sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=review_body.replace(',', ' '))['Body'].read().decode("utf-8").strip())
             for review_body in test['review_body']]

In [ ]:
tp = 0
fn = 0
tn = 0
fp = 0

# TODO:  Compare to test['is_positive_sentiment']
for index, response in enumerate(responses):
    if (test.iloc[index]['is_positive_sentiment'] == 1):
        if response == 1:
            # True positive
            tp = tp + 1
        else:
            # False negative
            fn = fn + 1
    else:
        if (test.iloc[index]['is_positive_sentiment'] == 0):
            # True negative
            tn = tn + 1
        else:
            # False positive
            fp = fp + 1

In [ ]:
#Confusion matrix
print ("%d %d" % (tn, fp))
print ("%d %d" % (fn, tp))

accuracy  = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall    = tn / (tn + fn)
f1        = (2 * precision * recall)/(precision + recall)

print ("accuracy:\t%.4f\nprecision:\t%.4f\nrecall:\t\t%.4f\nf1:\t\t%.4f" % (accuracy, precision, recall, f1))